# Importing libraries and setting up client

In [2]:
import pymongo
from pymongo import MongoClient
import praw
from praw.models import MoreComments
import pandas as pd
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords


client = MongoClient('mongodb://localhost:27017/')
db = client["reddit"]

In [3]:
reddit = praw.Reddit(client_id = '_Ij9InJJ1MqC-Q', client_secret = '9XF46Dd9icSEmAghFivwt9J9uGU', user_agent = 'Reddit WebScraping', username = 'anushkanarula01', password = '')
subreddit = reddit.subreddit('india')

# Scraping Reddit and Saving the data to MongoDB database

In [ ]:
#List of all the flairs. These will be the keys in classification:
flairs = ["AskIndia", "Non-Political", "[R]eddiquette", "Scheduled", "Photography", "Science/Technology","Politics", "Business/Finance", "Policy/Economy", "Sports", "Food", "AMA"]

for flair in flairs:
    #The posts' data is collected by searching by the flair name in the list. Top 100 posts are recorded and stored for analysis.
    relevant_subreddits = subreddit.search(f"flair_name:{flair}",limit=100)
    for submission in relevant_subreddits:
        posts = {
        "title":str(submission.title),
        "score":str(submission.score),
        "id":str(submission.id),
        "url":str(submission.url),
        "comms_num":str(submission.num_comments),
        "created":str(submission.created),
        "body":str(submission.selftext),
        "author":str(submission.author),
        "flair":str(flair),	
        }

#  Only top ten comments and their authors are considered. 
        submission.comments.replace_more(limit=None)
        comment = ''
        authors = ''
        count = 0
        for top_level_comment in submission.comments:
            comment = comment + ' ' + top_level_comment.body
            authors = authors + ' ' + str(top_level_comment.author)
            count+=1     
            if(count > 10):
                break

        posts["comment"] = str(comment)
        posts["authors"] = str(authors)
        result = db.posts.insert_one(posts)

# Text Cleaning 

In [7]:
replace_by_space = re.compile('[/(){}\[\]\|@,;]')
replace_symbol = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = replace_by_space.sub(' ', text) # replacing certain symbols by space in text
    text = replace_symbol.sub('', text) # deleting symbols from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # removing STOPWORDS from text
    return text


# Text Pre-Processing and Saving the final data into a CSV file

In [6]:
collection = db.posts
posts = pd.DataFrame(list(collection.find()))

del posts['_id']

posts['title'] = posts['title'].apply(str())
posts['body'] = posts['body'].apply(str())
posts['comment'] = posts['comment'].apply(str())

posts['title'] = posts['title'].apply(clean_text)
posts['body'] = posts['body'].apply(clean_text)
posts['comment'] = posts['comment'].apply(clean_text)

combined_features = posts["title"] + posts["comment"] + posts["url"] + posts["body"]
posts = posts.assign(combined_features = combined_features)

#saving the csv file
posts.to_csv('data.csv',index=False)